In [1]:
from mavis import *

hdul = fits.open('data/psd_turb.fits')
psd_turb_data = np.asarray(hdul[0].data, np.float64)
hdul = fits.open('data/res_noise_ngs.fits')
res_noise_ngs_data = np.asarray(hdul[0].data, np.float64)
res_noise_ngs_tip = res_noise_ngs_data[0]
res_noise_ngs_tilt = res_noise_ngs_data[1]
minTipIdx = np.where(res_noise_ngs_tip == np.amin(res_noise_ngs_tip))
minTiltIdx = np.where(res_noise_ngs_tilt == np.amin(res_noise_ngs_tilt))
print(minTipIdx[0], res_noise_ngs_tip[minTipIdx[0][0]])
print(minTiltIdx[0], res_noise_ngs_tilt[minTiltIdx[0][0]])
hdul = fits.open('data/Cn.fits')
Cnn_data = np.asarray(hdul[0].data, np.float64)
print(Cnn_data)
hdul = fits.open('data/minim_wind_noise12071nm2_psdmavis.fits')
windNoise = np.asarray(hdul[0].data, np.float64)
minTipIdx = np.where(windNoise[0] == np.amin(windNoise[0]))
minTiltIdx = np.where(windNoise[1] == np.amin(windNoise[1]))
print(minTipIdx[0], minTipIdx[1], windNoise[0][minTipIdx[0][0], minTipIdx[1][0]])
print(minTiltIdx[0], minTiltIdx[1], windNoise[1][minTiltIdx[0][0], minTiltIdx[1][0]])

[13] 2082.41162109375
[10] 1351.4102783203125
[[2369.9699707     0.            0.            0.            0.
     0.        ]
 [   0.         1538.56994629    0.            0.            0.
     0.        ]
 [   0.            0.         2082.40991211    0.            0.
     0.        ]
 [   0.            0.            0.         1351.41003418    0.
     0.        ]
 [   0.            0.            0.            0.         2916.64990234
     0.        ]
 [   0.            0.            0.            0.            0.
  1906.07995605]]
[98] [0] 71.33271789550781
[98] [0] 60.46025848388672


In [2]:
# phi_noise e' una costante pari a sigma^2 cog # / Df 
hdul = fits.open('data/windpsd_mavis.fits')
psd_data = np.asarray(hdul[0].data, np.float64)
psd_freq = np.asarray(np.linspace(0.5, 250.0, 500))
psd_tip_wind = np.zeros((500))
psd_tilt_wind = np.zeros((500))
psd_tip_wind[0:200] = psd_data[1,:]
psd_tilt_wind[0:200] = psd_data[2,:]
var1x = 0.05993281522281573 * pixel_scale**2
bias = 0.4300779971881394
print(computeNoiseResidual(0.25, 250.0, 1000, var1x, bias, gpulib ))
# queste due varianze vanno in diagonale di Cn (insieme a quelle delle altre 2 stelle, da calcolare)
# cabia il flusso, strhel fwhw, calcolo di bias diversi e ripetizione di questo calcolo
print(computeWindResidual(psd_freq, psd_tip_wind, psd_tilt_wind, var1x, bias, gpulib ))
# queste due varianze vanno in diagonale di C1
# questo conto e' fatto una volta sola , sulla stella piu' brillante

(array(2108.85526983), array(1361.73987558))
(array(71.89646223), array(61.02400116))


In [3]:
%%timeit
computeNoiseResidual(0.25, 250.0, 1000, var1x, bias, cpulib )

303 ms ± 1.04 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%%timeit
computeNoiseResidual(0.25, 250.0, 1000, var1x, bias, gpulib )

174 ms ± 149 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%%timeit
computeWindResidual(psd_freq, psd_tip_wind, psd_tilt_wind, var1x, bias, cpulib )

8.28 s ± 33.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit
computeWindResidual(psd_freq, psd_tip_wind, psd_tilt_wind, var1x, bias, gpulib )

182 ms ± 2.83 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
